<a href="https://colab.research.google.com/github/TusharPaul01/ChatBot-ML-LSTM-/blob/main/JUIT_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from getpass import getpass
# uplink_key = getpass('Enter your Uplink key: ')

uplink_key = 'server_TFKWGAGVLZ3VWY6T5KD6KWBV-7XQ4BDMT2GO2DEOD'

In [2]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=a5c5c182c09816a92b75d3a072cf3471e924974ca7a8d0f3b54964b71416cd75
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [3]:
import anvil.server
anvil.server.connect('server_TFKWGAGVLZ3VWY6T5KD6KWBV-7XQ4BDMT2GO2DEOD')


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "ChatBot" as SERVER


In [4]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [5]:
# load data from JSON file
with open('intents.json') as file:
    data = json.load(file)

# extract text and intent from data
texts = []
intents = []
for intent in data['intents']:
    for text in intent['text']:
        texts.append(text)
        intents.append(intent['intent'])

# tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
encoded_texts = tokenizer.texts_to_sequences(texts)

# save tokenizer
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# pad sequences to have equal length
max_len = max([len(x) for x in encoded_texts])
padded_texts = pad_sequences(encoded_texts, maxlen=max_len, padding='post')

# create label encoder object
le = LabelEncoder()

# fit and transform the intents to integer labels
encoded_intents = le.fit_transform(intents)

# get the number of unique labels
num_intents = len(le.classes_)

# apply one-hot encoding to the integer labels
encoded_intents = tf.one_hot(encoded_intents, depth=num_intents)

# define model architecture
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_len)(input_layer)
lstm_layer = LSTM(128)(embedding_layer)
output_layer = Dense(num_intents, activation='softmax')(lstm_layer)
model = Model(inputs=input_layer, outputs=output_layer)

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(padded_texts, encoded_intents, epochs=50, batch_size=16)

# save model
model.save('chatbot_model34.h5')


Epoch 1/50
26/26 [==============================] - 4s 15ms/step - loss: 3.6190 - accuracy: 0.0519
Epoch 2/50
26/26 [==============================] - 0s 15ms/step - loss: 3.5178 - accuracy: 0.0667
Epoch 3/50
26/26 [==============================] - 0s 16ms/step - loss: 3.3207 - accuracy: 0.1210
Epoch 4/50
26/26 [==============================] - 0s 15ms/step - loss: 2.8070 - accuracy: 0.1852
Epoch 5/50
26/26 [==============================] - 1s 23ms/step - loss: 2.4014 - accuracy: 0.2741
Epoch 6/50
26/26 [==============================] - 1s 28ms/step - loss: 2.0073 - accuracy: 0.3951
Epoch 7/50
26/26 [==============================] - 1s 32ms/step - loss: 1.5839 - accuracy: 0.5358
Epoch 8/50
26/26 [==============================] - 1s 24ms/step - loss: 1.2500 - accuracy: 0.6074
Epoch 9/50
26/26 [==============================] - 1s 27ms/step - loss: 0.9588 - accuracy: 0.7284
Epoch 10/50
26/26 [==============================] - 1s 26ms/step - loss: 0.7251 - accuracy: 0.8148
Epoch 11/

In [6]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# load data from JSON file
with open('intents_final.json') as file:
    data = json.load(file)

# extract text and intent from data
texts = []
intents = []
for intent in data['intents']:
    for text in intent['text']:
        texts.append(text)
        intents.append(intent['intent'])

# tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# load saved model
model = load_model('chatbot_model34.h5')

# define maximum sequence length
max_len = model.input_shape[1]

# create label encoder object
le = LabelEncoder()
le.fit(intents)

# create inverse mapping of label encoder for intent prediction
intent_mapping = {i: label for i, label in enumerate(le.classes_)}

# start chatbot interaction


In [7]:
@anvil.server.callable
def chatbot(sentence):
  while True:
    # get user input
      user_input = sentence.lower().strip()

      # check if user wants to quit
      if user_input == 'quit':
        break

      # encode user input text
      encoded_input = tokenizer.texts_to_sequences([user_input])
      padded_input = pad_sequences(encoded_input, maxlen=max_len, padding='post')

      # predict intent
      intent_prob = model.predict(padded_input)[0]
      intent_idx = np.argmax(intent_prob)
      intent_label = le.inverse_transform([intent_idx])[0]

      # retrieve response
      for intent in data['intents']:
        if intent['intent'] == intent_label:
            response = np.random.choice(intent['responses'])
            return response
            break


In [ ]:
anvil.server.wait_forever()

1/1 [==============================] - 0s 22ms/step
